In [1]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup 
import re 
from nltk.corpus import stopwords



In [2]:

train = pd.read_csv("../input/labeledTrainData.tsv", header=0,\
                    delimiter="\t", quoting=3)

test = pd.read_csv("../input/testData.tsv",header=0,\
                    delimiter="\t", quoting=3)

In [21]:
train.head()

,id,sentiment,review
0,"""5814_8""",1,"""With all this stuff going down at the moment ..."
1,"""2381_9""",1,"""\""The Classic War of the Worlds\"" by Timothy ..."
2,"""7759_3""",0,"""The film starts with a manager (Nicholas Bell..."
3,"""3630_4""",0,"""It must be assumed that those who praised thi..."
4,"""9495_8""",1,"""Superbly trashy and wondrously unpretentious ..."


In [3]:
# This function converts a text to a sequence of words.
def review_wordlist(review, remove_stopwords=False):
    # Remove html tags
    review_text = BeautifulSoup(review).get_text()
    # Remove non-letter.
    review_text = re.sub("[^a-zA-Z]"," ",review_text)
    # Lower case and splitting
    words = review_text.lower().split()
    # Optionally remove stopwords
    if remove_stopwords:
        stops = set(stopwords.words("english"))     
        words = [w for w in words if not w in stops]
    
    return(words)

In [4]:


import nltk.data

tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')


In [5]:
# This function splits into sentences
def review_sentences(review, tokenizer, remove_stopwords=False):
    raw_sentences = tokenizer.tokenize(review.strip())
    sentences = []
    for raw_sentence in raw_sentences:
        if len(raw_sentence)>0:
            sentences.append(review_wordlist(raw_sentence,\
                                            remove_stopwords))

    return sentences


In [6]:

sentences = []
for review in train["review"]:
    sentences += review_sentences(review, tokenizer)
    

Parsing sentences from training set


/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:219: UserWarning: "b'.'" looks like a filename, not markup. You should probably open this file and pass the filehandle into Beautiful Soup.
  ' Beautiful Soup.' % markup)
/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:282: UserWarning: "http://www.happierabroad.com"" looks like a URL. Beautiful Soup is not an HTTP client. Yo

In [8]:
#selecting various features
num_features = 300 
min_word_count = 40 
num_workers = 4     
context = 10        
downsampling = 1e-3 


from gensim.models import word2vec
print("Training model....")
model = word2vec.Word2Vec(sentences,
                          workers=num_workers,
                          size=num_features,
                          sample=downsampling,
                          min_count=min_word_count,
                          window=context
                         )




model.save(model_name)


2020-08-19 10:30:09,005 : INFO : 'pattern' package not found; tag filters are not available for English
2020-08-19 10:30:09,016 : INFO : collecting all words and their counts
2020-08-19 10:30:09,017 : INFO : PROGRESS: at sentence #0, processed 0 words, keeping 0 word types
2020-08-19 10:30:09,092 : INFO : PROGRESS: at sentence #10000, processed 225803 words, keeping 17776 word types
2020-08-19 10:30:09,172 : INFO : PROGRESS: at sentence #20000, processed 451892 words, keeping 24948 word types


Training model....


2020-08-19 10:30:09,251 : INFO : PROGRESS: at sentence #30000, processed 671315 words, keeping 30034 word types
2020-08-19 10:30:09,330 : INFO : PROGRESS: at sentence #40000, processed 897815 words, keeping 34348 word types
2020-08-19 10:30:09,410 : INFO : PROGRESS: at sentence #50000, processed 1116963 words, keeping 37761 word types
2020-08-19 10:30:09,490 : INFO : PROGRESS: at sentence #60000, processed 1338404 words, keeping 40723 word types
2020-08-19 10:30:09,573 : INFO : PROGRESS: at sentence #70000, processed 1561580 words, keeping 43333 word types
2020-08-19 10:30:09,657 : INFO : PROGRESS: at sentence #80000, processed 1780887 words, keeping 45714 word types
2020-08-19 10:30:09,738 : INFO : PROGRESS: at sentence #90000, processed 2004996 words, keeping 48135 word types
2020-08-19 10:30:09,818 : INFO : PROGRESS: at sentence #100000, processed 2226966 words, keeping 50207 word types
2020-08-19 10:30:09,896 : INFO : PROGRESS: at sentence #110000, processed 2446580 words, keeping 

2020-08-19 10:30:57,416 : INFO : PROGRESS: at 98.91% examples, 439172 words/s, in_qsize 7, out_qsize 0
2020-08-19 10:30:57,773 : INFO : worker thread finished; awaiting finish of 3 more threads
2020-08-19 10:30:57,783 : INFO : worker thread finished; awaiting finish of 2 more threads
2020-08-19 10:30:57,791 : INFO : worker thread finished; awaiting finish of 1 more threads
2020-08-19 10:30:57,799 : INFO : worker thread finished; awaiting finish of 0 more threads
2020-08-19 10:30:57,800 : INFO : training on 29603620 raw words (20222939 effective words) took 45.9s, 440238 effective words/s
2020-08-19 10:30:57,802 : INFO : precomputing L2-norms of word weight vectors
2020-08-19 10:30:57,906 : INFO : saving Word2Vec object under 300features_40minwords_10context, separately None
2020-08-19 10:30:57,908 : INFO : not storing attribute syn0norm
2020-08-19 10:30:57,910 : INFO : not storing attribute cum_table
2020-08-19 10:30:58,345 : INFO : saved 300features_40minwords_10context


In [22]:
model.wv.most_similar("good")


[('decent', 0.6500996947288513),
 ('bad', 0.6421070098876953),
 ('great', 0.6385183334350586),
 ('nice', 0.5846697092056274),
 ('fine', 0.5736824870109558),
 ('cool', 0.5378494262695312),
 ('funny', 0.5323127508163452),
 ('ok', 0.520061731338501),
 ('interesting', 0.5074468851089478),
 ('poor', 0.4989340007305145)]

In [13]:
model.wv.syn0.shape

(8306, 300)

In [14]:
# Function to average all word vectors
def featureVecMethod(words, model, num_features):
    featureVec = np.zeros(num_features,dtype="float32")
    nwords = 0
    
    index2word_set = set(model.wv.index2word)
    
    for word in  words:
        if word in index2word_set:
            nwords = nwords + 1
            featureVec = np.add(featureVec,model[word])
    
    # Divide the result by number of words 
    featureVec = np.divide(featureVec, nwords)
    return featureVec



In [15]:
# Function to calculate the average feature vector
def getAvgFeatureVecs(reviews, model, num_features):
    counter = 0
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    for review in reviews:
            
        reviewFeatureVecs[counter] = featureVecMethod(review, model, num_features)
        counter = counter+1
        
    return reviewFeatureVecs


In [16]:
# Calculating average feature vector for training set
clean_train_reviews = []
for review in train['review']:
    clean_train_reviews.append(review_wordlist(review, remove_stopwords=True))
    
trainDataVecs = getAvgFeatureVecs(clean_train_reviews, model, num_features)

/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [17]:
# Calculating average feature vactors for test set     
clean_test_reviews = []
for review in test["review"]:
    clean_test_reviews.append(review_wordlist(review,remove_stopwords=True))
    
testDataVecs = getAvgFeatureVecs(clean_test_reviews, model, num_features)

/opt/conda/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /opt/conda/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Review 0 of 25000


/opt/conda/lib/python3.6/site-packages/ipykernel_launcher.py:13: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  del sys.path[0]


Review 1000 of 25000
Review 2000 of 25000
Review 3000 of 25000
Review 4000 of 25000
Review 5000 of 25000
Review 6000 of 25000
Review 7000 of 25000
Review 8000 of 25000
Review 9000 of 25000
Review 10000 of 25000
Review 11000 of 25000
Review 12000 of 25000
Review 13000 of 25000
Review 14000 of 25000
Review 15000 of 25000
Review 16000 of 25000
Review 17000 of 25000
Review 18000 of 25000
Review 19000 of 25000
Review 20000 of 25000
Review 21000 of 25000
Review 22000 of 25000
Review 23000 of 25000
Review 24000 of 25000


In [18]:
from sklearn.model_selection import train_test_split
y=train["sentiment"]
trainDataVecs_train,trainDataVecs_val,y_train,y_val=train_test_split(trainDataVecs,y, test_size=0.2, random_state=42)

In [19]:
# Fitting a random forest classifier to the training data
from sklearn.ensemble import RandomForestClassifier
forest = RandomForestClassifier(n_estimators = 100)
    
print("Fitting random forest to training data....")    
forest = forest.fit(trainDataVecs_train, y_train)
    

Fitting random forest to training data....


In [20]:
from sklearn.metrics import roc_auc_score


trainprobsval = forest.predict_proba(trainDataVecs_val)


roc_auc_score(y_val, trainprobsval[:, 1])

0.90523180618912547